In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import csv
import pickle

In [5]:
#SET DATE FILTERS

prevalence = ['2020-06-21','2020-06-20','2020-06-19','2020-06-18','2020-06-17','2020-06-16','2020-06-15','2020-06-14','2020-06-13','2020-06-12',
              '2020-06-11','2020-06-10','2020-06-09','2020-06-08']

day7_current = ['2020-06-21','2020-06-20','2020-06-19','2020-06-18','2020-06-17','2020-06-16','2020-06-15',]

day7_previous = ['2020-06-14','2020-06-13','2020-06-12',
              '2020-06-11','2020-06-10','2020-06-09','2020-06-08']

curr_test_start = [int(20200621)]
curr_test_end = [int(20200615)]
prev_test_start = [int(20200614)]
prev_test_end = [int(20200608)]

In [6]:
#Bring in state level NYT Data
nytimes_state = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv"
state= pd.read_csv(nytimes_state)
print(state.head())

pop_df = pd.read_csv("state_base.csv"
                )

print(pop_df.head())

         date       state  fips  cases  deaths
0  2020-01-21  Washington    53      1       0
1  2020-01-22  Washington    53      1       0
2  2020-01-23  Washington    53      1       0
3  2020-01-24    Illinois    17      1       0
4  2020-01-24  Washington    53      1       0
        State state_code  cc_ee's  state_pop  CAE  Cable Stores  \
0     Alabama         AL      407    4908621   65            40   
1      Alaska         AK        0    3038999    0             0   
2     Arizona         AZ      744    7378494  237            17   
3    Arkansas         AR      117    3038999    1            13   
4  California         CA     5022   39937489    0           491   

   Business Services  Tech Ops  comcast_state  
0                  9       178              1  
1                  0         0              0  
2                  5        87              1  
3                  7        64              1  
4                219      1997              1  


In [7]:
#Sort and create diff fields 

state['State'] = state['state']
state.sort_values(by=['fips','date'],inplace=True, ascending=True)
print(state.head())

#Take difference between rows for new case and new death numbers
state['case_diff'] = state.cases.diff()
state['death_diff'] = state.deaths.diff()
print(state.head())

           date    state  fips  cases  deaths    State
586  2020-03-13  Alabama     1      6       0  Alabama
637  2020-03-14  Alabama     1     12       0  Alabama
689  2020-03-15  Alabama     1     23       0  Alabama
742  2020-03-16  Alabama     1     29       0  Alabama
795  2020-03-17  Alabama     1     39       0  Alabama
           date    state  fips  cases  deaths    State  case_diff  death_diff
586  2020-03-13  Alabama     1      6       0  Alabama        NaN         NaN
637  2020-03-14  Alabama     1     12       0  Alabama        6.0         0.0
689  2020-03-15  Alabama     1     23       0  Alabama       11.0         0.0
742  2020-03-16  Alabama     1     29       0  Alabama        6.0         0.0
795  2020-03-17  Alabama     1     39       0  Alabama       10.0         0.0


In [8]:
##CREATE State Prevalence measure
#Filter for past 14 days
state_range_14 = state[state['date'].isin(prevalence)]
print(state_range_14.head())

            date    state  fips  cases  deaths    State  case_diff  death_diff
5349  2020-06-08  Alabama     1  20925     718  Alabama      425.0        26.0
5404  2020-06-09  Alabama     1  21422     729  Alabama      497.0        11.0
5459  2020-06-10  Alabama     1  21989     744  Alabama      567.0        15.0
5514  2020-06-11  Alabama     1  22845     755  Alabama      856.0        11.0
5569  2020-06-12  Alabama     1  23710     769  Alabama      865.0        14.0


In [9]:
#create state group for prevalence

state_prevalence_grouped = state_range_14.groupby(['State'], as_index=False).sum()

state_prevalence_grouped['state_active'] = state_prevalence_grouped['case_diff']
print(state_prevalence_grouped.head())

        State  fips    cases  deaths  case_diff  death_diff  state_active
0     Alabama    14   358381   10919     9521.0       147.0        9521.0
1      Alaska    28    10264     135      271.0         2.0         271.0
2     Arizona    56   532527   16887    25742.0       302.0       25742.0
3    Arkansas    70   176123    2622     6135.0        71.0        6135.0
4  California    84  2175865   71799    46810.0       864.0       46810.0


In [10]:
#Take Current and previous rolling averages
state_range_current7 = state[state['date'].isin(day7_current)]
state_range_prev7 = state[state['date'].isin(day7_previous)]

print(state_range_current7.head(8))

            date    state  fips  cases  deaths    State  case_diff  death_diff
5734  2020-06-15  Alabama     1  26272     774  Alabama      657.0         1.0
5789  2020-06-16  Alabama     1  26912     785  Alabama      640.0        11.0
5844  2020-06-17  Alabama     1  27312     790  Alabama      400.0         5.0
5899  2020-06-18  Alabama     1  28206     810  Alabama      894.0        20.0
5954  2020-06-19  Alabama     1  29002     822  Alabama      796.0        12.0
6009  2020-06-20  Alabama     1  29549     838  Alabama      547.0        16.0
6064  2020-06-21  Alabama     1  30021     839  Alabama      472.0         1.0
5735  2020-06-15   Alaska     2    738      10   Alaska        4.0         0.0


In [11]:
#Groupby Means
#Group by with mean
state_current_grouped = state_range_current7.groupby(['State'],as_index=False).mean()
state_previous_grouped = state_range_prev7.groupby(['State'], as_index=False).mean()

state_current_grouped['state_curr7_case'] = state_current_grouped['case_diff']
state_previous_grouped['state_prev7_case'] = state_previous_grouped['case_diff']
state_current_grouped['state_curr7_death'] = state_current_grouped['death_diff']
state_previous_grouped['state_prev7_death'] = state_previous_grouped['death_diff']

print(state_current_grouped.head())
print(state_previous_grouped.head())

        State  fips          cases       deaths    case_diff  death_diff  \
0     Alabama   1.0   28182.000000   808.285714   629.428571    9.428571   
1      Alaska   2.0     797.857143    10.000000    17.857143    0.000000   
2     Arizona   4.0   44424.428571  1284.285714  2387.571429   22.285714   
3    Arkansas   5.0   14139.428571   205.428571   437.142857    6.571429   
4  California   6.0  167116.142857  5343.000000  3693.428571   61.142857   

   state_curr7_case  state_curr7_death  
0        629.428571           9.428571  
1         17.857143           0.000000  
2       2387.571429          22.285714  
3        437.142857           6.571429  
4       3693.428571          61.142857  
        State  fips          cases       deaths    case_diff  death_diff  \
0     Alabama   1.0   23015.285714   751.571429   730.714286   11.571429   
1      Alaska   2.0     668.428571     9.285714    20.857143    0.285714   
2     Arizona   4.0   31650.857143  1128.142857  1289.857143   20.857

In [14]:
state_active= pd.merge(pop_df,
                 state_prevalence_grouped[['State','state_active',
                                       ]],
                 on='State', 
                how='left')


state_current_w_pop = pd.merge(state_active,
                 state_current_grouped[['State','state_curr7_case',
                                       'state_curr7_death']],
                 on='State', 
                how='left')


county_state_df = pd.merge(state_current_w_pop,
                 state_previous_grouped[['State','state_prev7_case','state_prev7_death']],
                 on='State', 
                how='left')
print(county_state_df.head(10))

                  State state_code  cc_ee's  state_pop   CAE  Cable Stores  \
0               Alabama         AL      407    4908621    65            40   
1                Alaska         AK        0    3038999     0             0   
2               Arizona         AZ      744    7378494   237            17   
3              Arkansas         AR      117    3038999     1            13   
4            California         CA     5022   39937489     0           491   
5              Colorado         CO     8452    5845526  1206           246   
6           Connecticut         CT     1270    3563077   306            60   
7              Delaware         DE     1132     982895   499            37   
8  District of Columbia         DC      360     720687     0            16   
9               Florida         FL     7451   21992985  2123           468   

   Business Services  Tech Ops  comcast_state  state_active  state_curr7_case  \
0                  9       178              1        9521.0 

In [15]:
#Create Indicators

county_state_df['state_curr7_case100k'] = ((county_state_df['state_curr7_case']/county_state_df['state_pop'])*100000)
county_state_df['state_curr7_death100k'] = ((county_state_df['state_curr7_death']/county_state_df['state_pop'])*100000)
county_state_df['state_prev7_case100k'] = ((county_state_df['state_prev7_case']/county_state_df['state_pop'])*100000)
county_state_df['state_prev7_death100k'] = ((county_state_df['state_prev7_death']/county_state_df['state_pop'])*100000)


county_state_df['state_roll7case_diff'] = ((county_state_df['state_curr7_case100k']-county_state_df['state_prev7_case100k'])/county_state_df['state_prev7_case100k'])
county_state_df['state_roll7death_diff'] = ((county_state_df['state_curr7_death100k']-county_state_df['state_prev7_death100k'])/county_state_df['state_prev7_death100k'])
county_state_df['state_prevalence_per100k'] = ((county_state_df['state_active']/county_state_df['state_pop'])*100000)

print(county_state_df.head())

        State state_code  cc_ee's  state_pop  CAE  Cable Stores  \
0     Alabama         AL      407    4908621   65            40   
1      Alaska         AK        0    3038999    0             0   
2     Arizona         AZ      744    7378494  237            17   
3    Arkansas         AR      117    3038999    1            13   
4  California         CA     5022   39937489    0           491   

   Business Services  Tech Ops  comcast_state  state_active  \
0                  9       178              1        9521.0   
1                  0         0              0         271.0   
2                  5        87              1       25742.0   
3                  7        64              1        6135.0   
4                219      1997              1       46810.0   

             ...             state_curr7_death  state_prev7_case  \
0            ...                      9.428571        730.714286   
1            ...                      0.000000         20.857143   
2            .

# COVID TRACKING DATA

In [16]:
#Bring in COVID TRACKING data
url_all = "https://covidtracking.com/api/v1/states/daily.csv"
testing_all = pd.read_csv(url_all)
testing = testing_all[['date','state','positiveIncrease','negativeIncrease', 'totalTestResultsIncrease']]
testing['state_code'] = testing['state']

print(testing.head())
print(testing.dtypes)

       date state  positiveIncrease  negativeIncrease  \
0  20200621    AK                12              1121   
1  20200621    AL               472              4683   
2  20200621    AR                 0                 0   
3  20200621    AS                 0                 0   
4  20200621    AZ              2592             11872   

   totalTestResultsIncrease state_code  
0                      1133         AK  
1                      5155         AL  
2                         0         AR  
3                         0         AS  
4                     14464         AZ  
date                         int64
state                       object
positiveIncrease             int64
negativeIncrease             int64
totalTestResultsIncrease     int64
state_code                  object
dtype: object


C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [17]:
curr_testing = testing[(testing['date']>=curr_test_end) & (testing['date']<=curr_test_start)]
curr_testing['curr_pos_%'] = (curr_testing['positiveIncrease']/curr_testing['totalTestResultsIncrease'])
print(curr_testing.head(10))

prev_testing = testing[(testing['date']>=prev_test_end) & (testing['date']<=prev_test_start)]
prev_testing['prev_pos_%'] = (prev_testing['positiveIncrease']/prev_testing['totalTestResultsIncrease'])
print(prev_testing.head())

       date state  positiveIncrease  negativeIncrease  \
0  20200621    AK                12              1121   
1  20200621    AL               472              4683   
2  20200621    AR                 0                 0   
3  20200621    AS                 0                 0   
4  20200621    AZ              2592             11872   
5  20200621    CA              4515             80329   
6  20200621    CO               162              5155   
7  20200621    CT                40              3733   
8  20200621    DC                36              1533   
9  20200621    DE                94              2464   

   totalTestResultsIncrease state_code  curr_pos_%  
0                      1133         AK    0.010591  
1                      5155         AL    0.091562  
2                         0         AR         NaN  
3                         0         AS         NaN  
4                     14464         AZ    0.179204  
5                     84844         CA    0.053215  
6

C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [18]:
#Group by with mean
curr_test_grouped = curr_testing.groupby(['state_code'],as_index=False).mean()
prev_test_grouped = prev_testing.groupby(['state_code'], as_index=False).mean()

print(curr_test_grouped.head())
print(prev_test_grouped.head())

  state_code        date  positiveIncrease  negativeIncrease  \
0         AK  20200618.0         13.428571       2070.000000   
1         AL  20200618.0        629.428571       6116.714286   
2         AR  20200618.0        377.285714       6086.285714   
3         AS  20200618.0          0.000000          0.000000   
4         AZ  20200618.0       2385.571429       9285.714286   

   totalTestResultsIncrease  curr_pos_%  
0               2083.428571    0.007055  
1               6746.142857    0.108375  
2               6463.571429    0.083086  
3                  0.000000         NaN  
4              11671.285714    0.203062  
  state_code        date  positiveIncrease  negativeIncrease  \
0         AK  20200611.0         16.714286       1206.571429   
1         AL  20200611.0        730.714286       4682.000000   
2         AR  20200611.0        439.285714       4805.857143   
3         AS  20200611.0          0.000000          0.000000   
4         AZ  20200611.0       1257.428571 

In [19]:

curr_merge = pd.merge(county_state_df,
                 curr_test_grouped[['state_code','curr_pos_%']],
                 on='state_code', 
                how='left')
print(curr_merge.head())

mam_v2 = pd.merge(curr_merge,
                 prev_test_grouped[['state_code','prev_pos_%']],
                 on='state_code', 
                how='left')


mam_v2['testing_diff'] = (mam_v2['curr_pos_%'] - mam_v2['prev_pos_%'])
print(mam_v2.head())

        State state_code  cc_ee's  state_pop  CAE  Cable Stores  \
0     Alabama         AL      407    4908621   65            40   
1      Alaska         AK        0    3038999    0             0   
2     Arizona         AZ      744    7378494  237            17   
3    Arkansas         AR      117    3038999    1            13   
4  California         CA     5022   39937489    0           491   

   Business Services  Tech Ops  comcast_state  state_active     ...      \
0                  9       178              1        9521.0     ...       
1                  0         0              0         271.0     ...       
2                  5        87              1       25742.0     ...       
3                  7        64              1        6135.0     ...       
4                219      1997              1       46810.0     ...       

   state_prev7_case  state_prev7_death  state_curr7_case100k  \
0        730.714286          11.571429             12.822921   
1         20.85714

# Create Scores

In [22]:
print(mam_v2.state_prevalence_per100k.describe())

print(mam_v2.state_roll7case_diff.describe())

print(mam_v2.testing_diff.describe())

print(mam_v2.state_roll7death_diff.describe())

count     51.000000
mean      88.470675
std       61.687031
min        8.917410
25%       49.266244
50%       77.504960
75%      114.066335
max      348.878782
Name: state_prevalence_per100k, dtype: float64
count    51.000000
mean      0.163309
std       0.434117
min      -0.500000
25%      -0.155834
50%       0.008550
75%       0.409092
max       1.353846
Name: state_roll7case_diff, dtype: float64
count    51.000000
mean          -inf
std            NaN
min           -inf
25%      -0.009612
50%      -0.000417
75%       0.015526
max       0.302734
Name: testing_diff, dtype: float64
count    50.000000
mean           inf
std            NaN
min      -1.000000
25%      -0.332895
50%      -0.110324
75%       0.065259
max            inf
Name: state_roll7death_diff, dtype: float64


In [28]:
#Active 100k >94.09 = 2, >39.49 = 1, >39.49 = 0
#Rolling - >0.15 = 2, 0.15 to -0.15, 1, >0.15 = 0
#Deaths - >>5 = 4, 0


def active(n):
    if 0<=n<=49.48: score = 0
    elif 49.49<=n<=114.05: score = 1
    elif 114.06<=n<=100000: score = 2
    else: score = 0
    return(score)



def roll(n):
    if -1<=n<=-0.10: score = 0
    elif -0.10<=n<=0.10: score = 1
    elif 0.16<=n<=100: score = 2
    else: score = 0
    return(score)

def fatal(n):
    if n>=0.05: score = 4
    else: score = 0
    return(score)

def test(n):
    if -2<=n<=-.01: score = 0
    elif -.011111<=n<=.01: score = 1
    elif 0.01111 <=n<= 100: score =2
    else: score = 0
    return(score)


#county_filtered['Active_100k_Score'] = county_filtered['active_cases_100k'].apply(active)
#county_filtered['State_Active_100k_Score'] = county_filtered['state_prevalence_per100k'].apply(active)
#county_filtered['County_Roll_Score'] = county_filtered['roll7case_diff'].apply(roll)
#county_filtered['State_Roll_Score'] = county_filtered['state_roll7case_diff'].apply(roll)
#county_filtered['County_Fatality_Roll_Score'] = county_filtered['roll7death_diff'].apply(fatal)
#county_filtered['State_Fatality_Roll_Score'] = county_filtered['state_roll7death_diff'].apply(fatal)
#county_filtered['State_Testing_Score'] = county_filtered['testing_diff'].apply(test)




In [24]:
mam_v2['State_Active_100k_Score'] = mam_v2['state_prevalence_per100k'].apply(active)
mam_v2['State_Roll_Score'] = mam_v2['state_roll7case_diff'].apply(roll)
mam_v2['State_Fatality_Roll_Score'] = mam_v2['state_roll7death_diff'].apply(fatal)
mam_v2['State_Testing_Score'] = mam_v2['testing_diff'].apply(test)


print(mam_v2.head())

        State state_code  cc_ee's  state_pop  CAE  Cable Stores  \
0     Alabama         AL      407    4908621   65            40   
1      Alaska         AK        0    3038999    0             0   
2     Arizona         AZ      744    7378494  237            17   
3    Arkansas         AR      117    3038999    1            13   
4  California         CA     5022   39937489    0           491   

   Business Services  Tech Ops  comcast_state  state_active  \
0                  9       178              1        9521.0   
1                  0         0              0         271.0   
2                  5        87              1       25742.0   
3                  7        64              1        6135.0   
4                219      1997              1       46810.0   

          ...           state_roll7case_diff  state_roll7death_diff  \
0         ...                      -0.138612              -0.185185   
1         ...                      -0.143836              -1.000000   
2    

In [25]:
mam_v2['State_Composite_Score'] =  (mam_v2['State_Active_100k_Score'] + mam_v2['State_Roll_Score']+mam_v2['State_Fatality_Roll_Score']+
                                    mam_v2['State_Testing_Score'])


In [26]:
state_df = mam_v2.query('comcast_state ==1')
print(state_df.head())

        State state_code  cc_ee's  state_pop   CAE  Cable Stores  \
0     Alabama         AL      407    4908621    65            40   
2     Arizona         AZ      744    7378494   237            17   
3    Arkansas         AR      117    3038999     1            13   
4  California         CA     5022   39937489     0           491   
5    Colorado         CO     8452    5845526  1206           246   

   Business Services  Tech Ops  comcast_state  state_active  \
0                  9       178              1        9521.0   
2                  5        87              1       25742.0   
3                  7        64              1        6135.0   
4                219      1997              1       46810.0   
5                709       763              1        2537.0   

           ...            state_roll7death_diff  state_prevalence_per100k  \
0          ...                        -0.185185                193.964863   
2          ...                         0.068493           

In [27]:
state_df.to_csv('State_RTW_622.csv') 